In [19]:
import httplib2
import argparse
import pandas as pd
import requests
import json
from pprint import pprint
from datetime import datetime, timedelta
from yaml import load, FullLoader
from pprint import pprint
from oauth2client.file import Storage
from oauth2client import tools
from oauth2client import client
from googleapiclient import discovery

In [20]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CLIENT_SECRET_FILE = 'client_secret_grants.json' 
APPLICATION_NAME = 'Google Sheets API Report'
credential_path = 'sheets.googleapis.com-report.json'

In [21]:
store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, parents=[tools.argparser])
flags = parser.parse_args([])
if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
print('Storing credentials to ' + credential_path)

Storing credentials to sheets.googleapis.com-report.json


In [22]:
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl = discoveryUrl)
spreadsheetId = '1CmQqTVuzqDs5WX7fNAlB43AvTsoOfYqIE266impvCJI'

In [23]:
# rangeName = 'partners dict!A1:C10'

In [24]:
# result = service.spreadsheets().values().get(spreadsheetId = spreadsheetId, range = rangeName).execute()
# values = result.get('values', [])

In [25]:
# pprint(values)

In [26]:
# headers = values[0]
# rows = values[1:]
# data_top = pd.DataFrame.from_records(rows, columns = headers)
# data_bottom = pd.read_csv('data/metrika_data.txt', sep = '\t')

In [27]:
# data_top.head()

In [28]:
# data_bottom.head()

In [29]:
# result = pd.merge(data_top, data_bottom, how='outer', on=['utm_source', 'utm_medium'])

In [30]:
# result.head()

## Параметры Яндекс метрики

In [31]:
config_file = open('config.yaml', 'r')
config = load(config_file, Loader=FullLoader)
token = config['token']
headers = {'Authorization': 'OAuth ' + token}
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
counter = '51583145'
startDate = '2020-11-01'
endDate = '2021-03-31'
dimensions = ['ym:s:date', 'ym:s:deviceCategory', 'ym:s:hasAdBlocker', 'ym:s:trafficSource']
metrics = ['ym:s:visits', 'ym:s:users', 'ym:s:pageviews']
metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)

In [32]:
f = open('data/report_params.yaml', 'r')
params = load(f, Loader=FullLoader)
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
# startDate = (datetime.now() - timedelta(days = params['reports']['dau']['days_ago'])).strftime('%Y-%m-%d')
# endDate = (datetime.now() - timedelta(days = 1)).strftime('%Y-%m-%d')
dimensions = params['reports']['dau']['dimensions']
metrics = params['reports']['dau']['metrics']
sortby = params['reports']['dau']['sort']
counter = params['metrika counter']

In [33]:
def make_request(**kwargs):
    """Запрос к API Яндекс.Метрики. Возвращает JSON ответа"""
    return requests.get(API_URL, headers=headers, params = kwargs).json()

In [34]:
def json_handler(data):
    """Преобразование ответа JSON в спикок листов"""
    report = []
    for line in data['data']:
        dimension_list = [x['name'] for x in line['dimensions']]
        metrics_list = line['metrics']
        report.append(dimension_list + metrics_list)
    return report

In [35]:
response = make_request(date1 = startDate, date2 = endDate, dimensions = dimensions, metrics = metrics, 
                    id = counter, sort = sortby, accuracy = 1)

In [36]:
data = json_handler(response)
data

KeyError: 'data'

In [37]:
range_data = params['reports']['dau']['range_name']
range_name = '{}!{}{}:{}{}'.format(range_data['list'], range_data['start'], 2, range_data['end'], len(data) + 1)

NameError: name 'data' is not defined

In [38]:
range_name

NameError: name 'range_name' is not defined

In [39]:
body = {'values': data}

NameError: name 'data' is not defined

In [40]:
pprint(body)

NameError: name 'body' is not defined

In [41]:
service.spreadsheets().values().update(spreadsheetId=spreadsheetId, range=range_name, valueInputOption='USER_ENTERED', body=body).execute()

NameError: name 'range_name' is not defined